In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2021-08-24T17:00:00,ITA,19,Sicilia,38.115697,13.362357,740,102,842,24105,...,deceduti: n. 1 del 23/08/21 - n. 7 del 22/08/2...,17.0,NaN,NaN,267164.0,0.0,2933253.0,2704881.0,ITG,ITG1
11504,2021-08-24T17:00:00,ITA,9,Toscana,43.769231,11.255889,385,41,426,11685,...,NaN,3.0,NaN,NaN,263113.0,4209.0,4244261.0,1546043.0,ITI,ITI1
11505,2021-08-24T17:00:00,ITA,10,Umbria,43.106758,12.388247,54,8,62,1857,...,"Si fa presente che 1 dei ricoveri NON UTI, non...",1.0,NaN,NaN,60981.0,0.0,1065329.0,646985.0,ITI,ITI2
11506,2021-08-24T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,4,0,4,130,...,NaN,0.0,NaN,NaN,11213.0,773.0,105702.0,57688.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,2021-08-24T17:00:00,19,Sicilia,740,102,842,24105,24947,801,1491,...,267164,5638134,2142993.0,17.0,267164.0,0.0,2933253.0,2704881.0,ITG,ITG1
11504,2021-08-24T17:00:00,9,Toscana,385,41,426,11685,12111,-86,537,...,267322,5790304,2644322.0,3.0,263113.0,4209.0,4244261.0,1546043.0,ITI,ITI1
11505,2021-08-24T17:00:00,10,Umbria,54,8,62,1857,1919,32,105,...,60981,1712314,421207.0,1.0,60981.0,0.0,1065329.0,646985.0,ITI,ITI2
11506,2021-08-24T17:00:00,2,Valle d'Aosta,4,0,4,130,134,-13,4,...,11986,163390,78536.0,0.0,11213.0,773.0,105702.0,57688.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-08-24', '2021-08-23')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-08-24T17:00:00,1,Piemonte,142,9,151,3358,3509,-73,240,...,370900,5873423,2207036.0,0.0,351271.0,19629.0,3306152.0,2567271.0,ITC,ITC1
1,2021-08-24T17:00:00,2,Valle d'Aosta,4,0,4,130,134,-13,4,...,11986,163390,78536.0,0.0,11213.0,773.0,105702.0,57688.0,ITC,ITC2
2,2021-08-24T17:00:00,3,Lombardia,329,45,374,11962,12336,35,518,...,865880,13295296,5086331.0,6.0,805366.0,60514.0,10163219.0,3132077.0,ITC,ITC4
3,2021-08-24T17:00:00,5,Veneto,188,51,239,12199,12438,-401,470,...,450264,10372091,2004830.0,4.0,437019.0,13245.0,6270178.0,4101913.0,ITH,ITH3
4,2021-08-24T17:00:00,6,Friuli Venezia Giulia,31,11,42,901,943,32,99,...,110012,2437829,746033.0,0.0,95106.0,14906.0,1981156.0,456673.0,ITH,ITH4
5,2021-08-24T17:00:00,7,Liguria,85,10,95,1899,1994,-32,188,...,108787,2067075,789125.0,0.0,108787.0,0.0,1525134.0,541941.0,ITC,ITC3
6,2021-08-24T17:00:00,8,Emilia-Romagna,411,54,465,14377,14842,110,472,...,407914,7809660,2006685.0,3.0,407561.0,353.0,5452948.0,2356712.0,ITH,ITH5
7,2021-08-24T17:00:00,9,Toscana,385,41,426,11685,12111,-86,537,...,267322,5790304,2644322.0,3.0,263113.0,4209.0,4244261.0,1546043.0,ITI,ITI1
8,2021-08-24T17:00:00,10,Umbria,54,8,62,1857,1919,32,105,...,60981,1712314,421207.0,1.0,60981.0,0.0,1065329.0,646985.0,ITI,ITI2
9,2021-08-24T17:00:00,11,Marche,55,11,66,2927,2993,47,120,...,109235,1430341,832448.0,0.0,109235.0,0.0,1262943.0,167398.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-08-23T17:00:00,1,Piemonte,143,12,155,3427,3582,-61,102,...,370660,5856179,2201913.0,1.0,351062.0,19598.0,3300194.0,2555985.0,ITC,ITC1
1,2021-08-23T17:00:00,2,Valle d'Aosta,5,0,5,142,147,0,0,...,11982,162688,78204.0,0.0,11210.0,772.0,105575.0,57113.0,ITC,ITC2
2,2021-08-23T17:00:00,3,Lombardia,320,43,363,11938,12301,55,200,...,865362,13253673,5076404.0,4.0,805062.0,60300.0,10149009.0,3104664.0,ITC,ITC4
3,2021-08-23T17:00:00,5,Veneto,186,48,234,12605,12839,176,331,...,449794,10326391,2002227.0,2.0,436540.0,13254.0,6259639.0,4066752.0,ITH,ITH3
4,2021-08-23T17:00:00,6,Friuli Venezia Giulia,34,11,45,866,911,-29,25,...,109913,2429978,744368.0,3.0,95027.0,14886.0,1976685.0,453293.0,ITH,ITH4
5,2021-08-23T17:00:00,7,Liguria,84,11,95,1931,2026,35,57,...,108599,2058400,786575.0,0.0,108599.0,0.0,1521074.0,537326.0,ITC,ITC3
6,2021-08-23T17:00:00,8,Emilia-Romagna,402,52,454,14278,14732,263,558,...,407449,7778869,2004424.0,5.0,407096.0,353.0,5440760.0,2338109.0,ITH,ITH5
7,2021-08-23T17:00:00,9,Toscana,366,41,407,11790,12197,-68,434,...,266785,5775547,2638370.0,3.0,262586.0,4199.0,4236438.0,1539109.0,ITI,ITI1
8,2021-08-23T17:00:00,10,Umbria,53,7,60,1827,1887,-71,7,...,60876,1703747,420584.0,1.0,60876.0,0.0,1062826.0,640921.0,ITI,ITI2
9,2021-08-23T17:00:00,11,Marche,44,13,57,2889,2946,10,34,...,109115,1427407,830921.0,3.0,109115.0,0.0,1260677.0,166730.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-08-24T17:00:00,1,Piemonte,142,9,151,3358,3509,-73,240,...,5873423,2207036.0,0.0,351271.0,19629.0,3306152.0,2567271.0,ITC,ITC1,4.341375e+06
1,2021-08-24T17:00:00,2,Valle d'Aosta,4,0,4,130,134,-13,4,...,163390,78536.0,0.0,11213.0,773.0,105702.0,57688.0,ITC,ITC2,1.255010e+05
2,2021-08-24T17:00:00,3,Lombardia,329,45,374,11962,12336,35,518,...,13295296,5086331.0,6.0,805366.0,60514.0,10163219.0,3132077.0,ITC,ITC4,1.010397e+07
3,2021-08-24T17:00:00,5,Veneto,188,51,239,12199,12438,-401,470,...,10372091,2004830.0,4.0,437019.0,13245.0,6270178.0,4101913.0,ITH,ITH3,4.907704e+06
4,2021-08-24T17:00:00,6,Friuli Venezia Giulia,31,11,42,901,943,32,99,...,2437829,746033.0,0.0,95106.0,14906.0,1981156.0,456673.0,ITH,ITH4,1.211357e+06
5,2021-08-24T17:00:00,7,Liguria,85,10,95,1899,1994,-32,188,...,2067075,789125.0,0.0,108787.0,0.0,1525134.0,541941.0,ITC,ITC3,1.543127e+06
6,2021-08-24T17:00:00,8,Emilia-Romagna,411,54,465,14377,14842,110,472,...,7809660,2006685.0,3.0,407561.0,353.0,5452948.0,2356712.0,ITH,ITH5,4.467118e+06
7,2021-08-24T17:00:00,9,Toscana,385,41,426,11685,12111,-86,537,...,5790304,2644322.0,3.0,263113.0,4209.0,4244261.0,1546043.0,ITI,ITI1,3.722729e+06
8,2021-08-24T17:00:00,10,Umbria,54,8,62,1857,1919,32,105,...,1712314,421207.0,1.0,60981.0,0.0,1065329.0,646985.0,ITI,ITI2,8.802850e+05
9,2021-08-24T17:00:00,11,Marche,55,11,66,2927,2993,47,120,...,1430341,832448.0,0.0,109235.0,0.0,1262943.0,167398.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-08-24T17:00:00,1,Piemonte,142,9,151,3358,3509,-73,240,...,2567271.0,ITC,ITC1,4.341375e+06,Piemonte,142,5824,9,628,99
1,2021-08-24T17:00:00,2,Valle d'Aosta,4,0,4,130,134,-13,4,...,57688.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,4,83,0,33,0
2,2021-08-24T17:00:00,3,Lombardia,329,45,374,11962,12336,35,518,...,3132077.0,ITC,ITC4,1.010397e+07,Lombardia,329,6369,45,1416,114
3,2021-08-24T17:00:00,5,Veneto,188,51,239,12199,12438,-401,470,...,4101913.0,ITH,ITH3,4.907704e+06,Veneto,188,6000,51,1000,0
4,2021-08-24T17:00:00,6,Friuli Venezia Giulia,31,11,42,901,943,32,99,...,456673.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,31,1277,11,175,0
5,2021-08-24T17:00:00,7,Liguria,85,10,95,1899,1994,-32,188,...,541941.0,ITC,ITC3,1.543127e+06,Liguria,85,1684,10,218,12
6,2021-08-24T17:00:00,8,Emilia-Romagna,411,54,465,14377,14842,110,472,...,2356712.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,411,6538,54,760,0
7,2021-08-24T17:00:00,9,Toscana,385,41,426,11685,12111,-86,537,...,1546043.0,ITI,ITI1,3.722729e+06,Toscana,385,5033,41,570,40
8,2021-08-24T17:00:00,10,Umbria,54,8,62,1857,1919,32,105,...,646985.0,ITI,ITI2,8.802850e+05,Umbria,54,684,8,91,36
9,2021-08-24T17:00:00,11,Marche,55,11,66,2927,2993,47,120,...,167398.0,ITI,ITI3,1.518400e+06,Marche,55,959,11,213,61


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-08-24T17:00:00,19,Sicilia,740,102,842,24105,24947,801,1491,...,10.92,776,370,19927,9061.0,11,679,7.48,0.03001,5.38
1,2021-08-24T17:00:00,9,Toscana,385,41,426,11685,12111,-86,537,...,7.19,-105,103,14757,5952.0,8,615,3.64,0.01442,7.18
2,2021-08-24T17:00:00,15,Campania,366,20,386,8634,9020,-350,531,...,3.65,-352,341,19013,2691.0,13,868,2.79,0.00918,7.64
3,2021-08-24T17:00:00,3,Lombardia,329,45,374,11962,12336,35,518,...,3.18,24,318,41623,9927.0,7,476,1.24,0.00513,8.57
4,2021-08-24T17:00:00,8,Emilia-Romagna,411,54,465,14377,14842,110,472,...,7.11,99,-86,30791,2261.0,3,352,1.53,0.01057,9.13
5,2021-08-24T17:00:00,5,Veneto,188,51,239,12199,12438,-401,470,...,5.10,-406,139,45700,2603.0,2,869,1.03,0.00958,9.17
6,2021-08-24T17:00:00,12,Lazio,524,68,592,16173,16765,-312,425,...,7.21,-299,77,0,0.0,6,731,inf,0.00725,6.34
7,2021-08-24T17:00:00,1,Piemonte,142,9,151,3358,3509,-73,240,...,1.43,-69,138,17244,5123.0,0,313,1.39,0.00553,8.54
8,2021-08-24T17:00:00,18,Calabria,125,10,135,3992,4127,71,234,...,5.92,67,-23,2588,2299.0,2,161,9.04,0.01216,3.94
9,2021-08-24T17:00:00,7,Liguria,85,10,95,1899,1994,-32,188,...,4.59,-32,131,8675,2550.0,2,218,2.17,0.01218,7.05


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Sicilia,102,832,934,10.92
1,Sardegna,22,182,204,10.78
2,Umbria,8,83,91,8.79
3,Abruzzo,13,164,177,7.34
4,Lazio,68,875,943,7.21
5,Toscana,41,529,570,7.19
6,Emilia-Romagna,54,706,760,7.11
7,Friuli Venezia Giulia,11,164,175,6.29
8,Calabria,10,159,169,5.92
9,Puglia,25,457,482,5.19


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))